# STT Conversion Code(ETRI)

In [ ]:
import os
import json
import base64
import csv
import urllib3

In [ ]:
def encode_audio(file_path: str) -> str:
    """
    오디오 파일을 Base64로 인코딩한다.
    
    :param file_path: str - 오디오 파일의 경로
    :return: str - Base64로 인코딩된 오디오 파일 내용
    """
    with open(file_path, "rb") as file:
        return base64.b64encode(file.read()).decode("utf8")

def call_api(audio_contents: str, language_code: str, access_key: str, api_url: str) -> dict:
    """
    API에 오디오 데이터를 보내고, 응답을 JSON 형태로 반환한다.
    
    :param audio_contents: str - Base64 인코딩된 오디오 내용
    :param language_code: str - 언어 코드 (예: 'korean')
    :param access_key: str - API 접근 키
    :param api_url: str - API URL
    :return: dict - API 응답 JSON
    """
    http = urllib3.PoolManager()
    
    request_json = {
        "argument": {
            "language_code": language_code,
            "audio": audio_contents
        }
    }

    response = http.request(
        "POST",
        api_url,
        headers={"Content-Type": "application/json; charset=UTF-8","Authorization": access_key},
        body=json.dumps(request_json)
    )
    
    return json.loads(response.data.decode('utf-8'))

def write_to_csv(file_path: str, data: list):
    """
    주어진 데이터를 CSV 파일에 작성한다.
    
    :param file_path: str - CSV 파일의 경로
    :param data: list - CSV 파일에 쓸 데이터 리스트
    """
    with open(file_path, 'w', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(["파일", "STT 결과"])
        for row in data:
            csv_writer.writerow(row)

def process_audio_files(audio_dir_path: str, language_code: str, access_key: str, api_url: str, csv_file_path: str):
    """
    지정된 디렉토리의 모든 오디오 파일을 처리하고 결과를 CSV 파일에 저장한다.
    
    :param audio_dir_path: str - 오디오 파일 디렉토리 경로
    :param language_code: str - 언어 코드 (예: 'korean')
    :param access_key: str - API 접근 키
    :param api_url: str - API URL
    :param csv_file_path: str - 결과를 저장할 CSV 파일의 경로
    """
    api_call_count = 0
    results = []

    for filename in sorted(os.listdir(audio_dir_path)):
        if filename.endswith(".wav"):
            audio_file_path = os.path.join(audio_dir_path, filename)
            audio_contents = encode_audio(audio_file_path)
            response_body = call_api(audio_contents, language_code, access_key, api_url)
            stt_result = response_body.get('return_object', {}).get('recognized', '')
            results.append([filename, stt_result])

            api_call_count += 1
            if api_call_count % 10 == 0:
                print(f"API가 {api_call_count}번 호출되었습니다.")
    
    write_to_csv(csv_file_path, results)

In [ ]:
# 예제 실행
open_api_url = "http://aiopen.etri.re.kr:8000/WiseASR/Recognition"
access_key = "YOUR_ACCESS_KEY"
audio_dir_path = "AUDIO_PATH"
language_code = "korean"
csv_file_path = "CSV_FILE_PATH"

process_audio_files(audio_dir_path, language_code, access_key, open_api_url, csv_file_path)